In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Amazonas - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_AM.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Amazonas - Produção de Cimento (t)', axis=1)
data

,Ano,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda,Amazonas - Consumo de Cimento (t)
0,2008,5.017229e+07,2.568805e+06,12.460706,4.349579e+07,0.760020,0.703356,5.483335e+08,2.546644e+09,1.874883e+10,...,29.241577,1002.673957,1407.801557,29.368351,23.012008,70.408467,1.658576,0.678960,0.674670,769.233000
1,2009,5.047410e+07,2.574930e+06,12.433926,4.345535e+07,0.761567,0.704503,5.302764e+08,2.679945e+09,2.055015e+10,...,30.906682,1002.901517,1393.526887,28.853124,22.769544,71.758162,1.685334,0.682960,0.672987,744.252000
2,2010,5.169307e+07,2.569006e+06,12.457785,4.379523e+07,0.762693,0.705545,5.046425e+08,2.891930e+09,2.342247e+10,...,35.544249,1003.401682,1311.661141,28.071885,22.643630,74.482804,1.537130,0.686918,0.671387,796.116000
3,2011,5.272696e+07,2.545977e+06,12.460974,4.398860e+07,0.763406,0.706485,4.854622e+08,3.091988e+09,2.604265e+10,...,39.206921,1003.835072,1276.962232,27.828830,22.427653,74.624578,1.468307,0.690831,0.669868,948.011000
4,2012,5.353767e+07,2.524602e+06,12.414889,4.403561e+07,0.763641,0.707029,4.867661e+08,3.189506e+09,2.708753e+10,...,40.620066,1004.045485,1269.809720,27.699098,22.391859,74.970536,1.457821,0.694062,0.668360,994.932000
5,2013,5.434368e+07,2.507099e+06,12.394100,4.415452e+07,0.763611,0.708365,5.225848e+08,3.093872e+09,2.523394e+10,...,39.498004,1004.283223,1266.017495,27.731222,22.404779,74.742115,1.485513,0.699234,0.667140,1036.606000
6,2014,5.487107e+07,2.472487e+06,12.359116,4.412824e+07,0.763162,0.708895,5.676413e+08,2.779340e+09,1.954361e+10,...,38.217511,1004.431965,1277.324904,28.055466,22.412378,73.347472,1.524622,0.702586,0.665819,771.567857
7,2015,5.527657e+07,2.446996e+06,12.336147,4.411441e+07,0.762342,0.709137,5.179392e+08,2.561731e+09,1.327596e+10,...,38.739113,1004.517645,1275.202120,28.150076,22.434009,72.952065,1.509930,0.705552,0.664387,746.867486
8,2016,5.561997e+07,2.436680e+06,12.339032,4.416689e+07,0.770069,0.713039,4.524144e+08,2.516859e+09,1.011370e+10,...,39.689203,1004.590070,1250.291696,27.936211,22.456139,73.785027,1.450843,0.716221,0.658519,739.457657
9,2017,5.581151e+07,2.432227e+06,12.358279,4.424802e+07,0.769075,0.713224,4.213689e+08,2.497710e+09,8.456309e+09,...,39.521434,1004.691076,1227.047962,27.873459,22.397053,73.760599,1.416718,0.717241,0.658930,687.211000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
0,5.017229e+07,2.568805e+06,12.460706,4.349579e+07,0.760020,0.703356,5.483335e+08,2.546644e+09,1.874883e+10,2.351319e+10,...,8.396886,29.241577,1002.673957,1407.801557,29.368351,23.012008,70.408467,1.658576,0.678960,0.674670
1,5.047410e+07,2.574930e+06,12.433926,4.345535e+07,0.761567,0.704503,5.302764e+08,2.679945e+09,2.055015e+10,2.446518e+10,...,8.420659,30.906682,1002.901517,1393.526887,28.853124,22.769544,71.758162,1.685334,0.682960,0.672987
2,5.169307e+07,2.569006e+06,12.457785,4.379523e+07,0.762693,0.705545,5.046425e+08,2.891930e+09,2.342247e+10,2.606035e+10,...,8.389077,35.544249,1003.401682,1311.661141,28.071885,22.643630,74.482804,1.537130,0.686918,0.671387
3,5.272696e+07,2.545977e+06,12.460974,4.398860e+07,0.763406,0.706485,4.854622e+08,3.091988e+09,2.604265e+10,2.845738e+10,...,8.376520,39.206921,1003.835072,1276.962232,27.828830,22.427653,74.624578,1.468307,0.690831,0.669868
4,5.353767e+07,2.524602e+06,12.414889,4.403561e+07,0.763641,0.707029,4.867661e+08,3.189506e+09,2.708753e+10,3.181492e+10,...,11.321769,40.620066,1004.045485,1269.809720,27.699098,22.391859,74.970536,1.457821,0.694062,0.668360
5,5.434368e+07,2.507099e+06,12.394100,4.415452e+07,0.763611,0.708365,5.225848e+08,3.093872e+09,2.523394e+10,3.629162e+10,...,12.008921,39.498004,1004.283223,1266.017495,27.731222,22.404779,74.742115,1.485513,0.699234,0.667140
6,5.487107e+07,2.472487e+06,12.359116,4.412824e+07,0.763162,0.708895,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,12.610217,38.217511,1004.431965,1277.324904,28.055466,22.412378,73.347472,1.524622,0.702586,0.665819
7,5.527657e+07,2.446996e+06,12.336147,4.411441e+07,0.762342,0.709137,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,13.139356,38.739113,1004.517645,1275.202120,28.150076,22.434009,72.952065,1.509930,0.705552,0.664387
8,5.561997e+07,2.436680e+06,12.339032,4.416689e+07,0.770069,0.713039,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,13.263561,39.689203,1004.590070,1250.291696,27.936211,22.456139,73.785027,1.450843,0.716221,0.658519
9,5.581151e+07,2.432227e+06,12.358279,4.424802e+07,0.769075,0.713224,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,12.821072,39.521434,1004.691076,1227.047962,27.873459,22.397053,73.760599,1.416718,0.717241,0.658930


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      744.252000
1      796.116000
2      948.011000
3      994.932000
4     1036.606000
5      771.567857
6      746.867486
7      739.457657
8      687.211000
9      350.177000
10     533.444000
11     595.320000
12     610.155000
13     640.274000
14            NaN
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
0,-1.823984,1.303637,1.392423,-1.881765,-1.489637,-1.645646,1.204752,-0.990234,0.166091,-1.487840,...,-1.373236,-2.227879,-2.010367,1.924851,2.625774,2.633383,-2.467290,1.708024,-1.621756,1.535111
1,-1.673536,1.416732,0.791547,-2.028261,-0.964358,-1.283213,0.820657,-0.546688,0.428911,-1.378556,...,-1.361444,-1.801387,-1.684250,1.716762,1.560169,1.369363,-1.408300,1.984284,-1.320949,1.248581
2,-1.065906,1.307356,1.326890,-0.796982,-0.582138,-0.954158,0.275392,0.158672,0.847995,-1.195437,...,-1.377109,-0.613544,-0.967461,0.523363,-0.055613,0.712946,0.729492,0.454191,-1.023245,0.976441
3,-0.550533,0.882147,1.398444,-0.096457,-0.340162,-0.657254,-0.132596,0.824346,1.230291,-0.920269,...,-1.383337,0.324594,-0.346368,0.017539,-0.558307,-0.412991,0.840731,-0.256354,-0.728908,0.717851
4,-0.146413,0.487481,0.364414,0.073831,-0.260467,-0.485225,-0.104860,1.148829,1.382743,-0.534838,...,0.077489,0.686550,-0.044825,-0.086726,-0.826622,-0.599598,1.112174,-0.364616,-0.485905,0.461287
5,0.255366,0.164315,-0.102045,0.504621,-0.270630,-0.063266,0.657046,0.830615,1.112297,-0.020933,...,0.418312,0.399151,0.295877,-0.142008,-0.760182,-0.532243,0.932952,-0.078717,-0.096908,0.253666
6,0.518259,-0.474755,-0.886983,0.409399,-0.423086,0.104286,1.615453,-0.215960,0.282052,0.569384,...,0.716551,0.071172,0.509040,0.022826,-0.089571,-0.492626,-0.161305,0.325049,0.155273,0.028744
7,0.720395,-0.945418,-1.402348,0.359293,-0.701403,0.180503,0.558230,-0.940033,-0.632425,0.943972,...,0.979001,0.204772,0.631829,-0.008119,0.106103,-0.379858,-0.471547,0.173370,0.378325,-0.214922
8,0.891571,-1.135894,-1.337608,0.549425,1.921611,1.413182,-0.835563,-1.089341,-1.093812,1.140087,...,1.040606,0.448123,0.735622,-0.371251,-0.336219,-0.264487,0.182007,-0.436659,1.180811,-1.213618
9,0.987051,-1.218110,-0.905767,0.843354,1.584260,1.471729,-1.495939,-1.153056,-1.335633,1.102599,...,0.821134,0.405152,0.880374,-0.710087,-0.466003,-0.572517,0.162840,-0.788968,1.257539,-1.143706


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.85940512, -0.72744224, -0.2362819 ,  0.98046156,
          0.47860509,  0.86449885, -1.0453709 ,  1.21859064,
         -0.99976018,  0.79428371,  1.15575749, -0.52559858,
          1.30260559, -0.55253046, -0.25130359,  1.38787176,
          1.47154451, -1.80649616,  1.09296291,  0.94281138,
          0.52836084,  0.61258103,  0.87656257,  1.47801536,
          1.71982086,  1.28221181,  1.39086472,  1.58209844,
          1.08872091,  0.55212962, -1.81398118,  1.44113674,
          1.11191094,  1.30470835,  0.86390516, -1.67365968,
         -1.70738939, -1.55813196, -0.48498301, -0.75714639,
          0.40770893, -0.27394734,  2.02323219,  1.2908951 ,
          1.70465513,  1.81148084,  1.39968653, -1.36191442,
          1.28024593,  1.11054795,  0.12662799,  1.40414112,
          1.40468999,  1.45643959,  1.58509336,  1.32283157,
          1.30222047,  1.55138602,  1.91801602, -0.77489784,
          2.32098884, -1.1942541 , -1.17081042,  0.66742486,
          1.17089806,  0

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      744.252000
1      796.116000
2      948.011000
3      994.932000
4     1036.606000
5      771.567857
6      746.867486
7      739.457657
8      687.211000
9      350.177000
10     533.444000
11     595.320000
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    595.32
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
12,0.548425,-0.389186,-0.34651,0.585399,-0.362055,0.429222,1.108405,2.635823,-0.189959,0.726405,...,0.598328,0.774435,0.860523,-1.989017,-1.013503,-0.547354,1.083075,-1.613773,0.941105,-1.177883


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.54842464, -0.38918635, -0.34651039,  0.5853988 ,
         -0.36205544,  0.4292222 ,  1.10840491,  2.63582309,
         -0.18995911,  0.72640489,  2.66833308, -0.032632  ,
          0.98802435,  0.24980494, -0.08955808,  1.85434394,
          2.02706568, -2.21150703,  1.42938591,  0.85132668,
          0.14092386,  0.30902324,  1.00267904,  2.27970137,
          1.11272593,  0.61007902,  0.89064538,  1.42231438,
          1.02650562,  1.47127593, -1.53201473,  1.73503858,
          1.66610613,  1.0456894 ,  0.55289259, -1.81216593,
         -2.10342296, -2.55377478,  0.41940628, -1.75756445,
          0.241023  , -0.98659345,  1.52184916,  1.01398607,
          1.49290967,  1.40764092,  1.87930411, -2.51892072,
          2.83356214,  1.58620924, -0.02524128,  1.54231542,
          1.5176119 ,  1.7355269 ,  1.87941982,  1.54198129,
          1.51276712,  1.83795594,  2.62210693,  1.04993174,
          0.96476493, -1.48277861, -2.26044738,  0.59832796,
          0.77443537,  0

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    610.155
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [19]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [20]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [22]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[226725554, 2167763140, 1370633320, 919828122, 3853491769, 600482674, 2658099782, 618609720, 4122091373, 444367665, 3211753369, 2507651597, 2176768502, 2892529902, 1878543206, 2375514204, 2213521910, 3074679713, 3038564230, 3700879752, 431483211, 3948184146, 2368103839, 2110927800, 3613737846, 3687026519, 2674829744, 877120933, 208711734, 512505705, 2561074269, 1736194591, 1222742080, 152588429, 881852679, 1422549384, 2182458486, 2425849179, 218007687, 3782180554, 348674819, 1577455332, 476156692, 2338034098, 393691804, 471289391, 465972034, 1767923179, 3827622208, 3237014049, 3757636949, 260566744, 3625564401, 2915588471, 1335643839, 2401123451, 3465284174, 144157813, 3389907174, 2996234055, 2187874191, 2143956513, 4063669984, 3122646892, 350212498, 2258844526, 3383609864, 1555291711, 1197021008, 2445684183, 2791741730, 2490748560, 2360203376, 3900817245, 2299779702, 4093310459, 4272552867, 1803176138, 369158374, 2007495774, 3337615154, 4014924678, 2427074830, 2996958690, 1922171341, 

loss: 427.8504333496094


Step: 89 ___________________________________________
loss: 366.0363464355469


Step: 90 ___________________________________________
loss: 420.89013671875


Step: 91 ___________________________________________
loss: 305.14630126953125


Step: 92 ___________________________________________
loss: 448.0955810546875


Step: 93 ___________________________________________
loss: 250.92193603515625


Step: 94 ___________________________________________
loss: 393.1033020019531


Step: 95 ___________________________________________
loss: 438.5193786621094


Step: 96 ___________________________________________
loss: 178.7506103515625


Step: 97 ___________________________________________
loss: 291.4132995605469


Step: 98 ___________________________________________
loss: 350.4915466308594


Step: 99 ___________________________________________
loss: 236.552490234375


final_seed: 2658099782


In [23]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 590395.1875
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 577546.2500
Epoch 3/10000
1/1 [==============================] - 0s 7ms/step - loss: 540176.7500
Epoch 4/10000
1/1 [==============================] - 0s 8ms/step - loss: 291430.9375
Epoch 5/10000
1/1 [==============================] - 0s 7ms/step - loss: 32443.0762
Epoch 6/10000
1/1 [==============================] - 0s 7ms/step - loss: 23403.9531
Epoch 7/10000
1/1 [==============================] - 0s 7ms/step - loss: 16364.6875
Epoch 8/10000
1/1 [==============================] - 0s 7ms/step - loss: 12554.0049
Epoch 9/10000
1/1 [==============================] - 0s 5ms/step - loss: 19609.4004
Epoch 10/10000
1/1 [==============================] - 0s 7ms/step - loss: 12364.6709
Epoch 11/10000
1/1 [==============================] - 0s 5ms/step - loss: 13960.4580
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 5ms/step - loss: 3217.4199
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 4850.2642
Epoch 100/10000
1/1 [==============================] - 0s 5ms/step - loss: 2099.9885
Epoch 101/10000
1/1 [==============================] - 0s 5ms/step - loss: 2908.8933
Epoch 102/10000
1/1 [==============================] - 0s 5ms/step - loss: 2512.3223
Epoch 103/10000
1/1 [==============================] - 0s 5ms/step - loss: 3213.8215
Epoch 104/10000
1/1 [==============================] - 0s 6ms/step - loss: 5284.9497
Epoch 105/10000
1/1 [==============================] - 0s 6ms/step - loss: 2209.4707
Epoch 106/10000
1/1 [==============================] - 0s 6ms/step - loss: 2430.5554
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - loss: 3033.4941
Epoch 108/10000
1/1 [==============================] - 0s 6ms/step - loss: 4646.0220
Epoch 109/10000
1/1 [==============================] - 0s 5ms/step - loss: 2654.82

1/1 [==============================] - 0s 4ms/step - loss: 3438.4265
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 1873.1250
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 3592.5095
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 4794.9043
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 2792.3118
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 1845.2240
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 2291.8577
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 4445.0620
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 1936.8364
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - loss: 3703.8938
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 4058.7229
Epoch 205/10000
1/1 [==============================] - 0s 5ms/step - loss: 3532.3

1/1 [==============================] - 0s 4ms/step - loss: 3606.5557
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 2452.3103
Epoch 292/10000
1/1 [==============================] - 0s 5ms/step - loss: 2586.1028
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 3212.6692
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 7103.9458
Epoch 295/10000
1/1 [==============================] - 0s 5ms/step - loss: 3014.9395
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 2185.1628
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 1580.0190
Epoch 298/10000
1/1 [==============================] - 0s 5ms/step - loss: 4039.3801
Epoch 299/10000
1/1 [==============================] - 0s 6ms/step - loss: 3023.6531
Epoch 300/10000
1/1 [==============================] - 0s 6ms/step - loss: 1681.4349
Epoch 301/10000
1/1 [==============================] - 0s 6ms/step - loss: 1190.9

1/1 [==============================] - 0s 4ms/step - loss: 2886.0684
Epoch 387/10000
1/1 [==============================] - 0s 5ms/step - loss: 2611.9724
Epoch 388/10000
1/1 [==============================] - 0s 5ms/step - loss: 1376.7360
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 1492.5337
Epoch 390/10000
1/1 [==============================] - 0s 5ms/step - loss: 4030.7449
Epoch 391/10000
1/1 [==============================] - 0s 5ms/step - loss: 1967.4342
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 1905.9902
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - loss: 1256.5073
Epoch 394/10000
1/1 [==============================] - 0s 5ms/step - loss: 3147.3274
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 2201.0676
Epoch 396/10000
1/1 [==============================] - 0s 5ms/step - loss: 2364.7197
Epoch 397/10000
1/1 [==============================] - 0s 5ms/step - loss: 4623.5

...

1/1 [==============================] - 0s 5ms/step - loss: 862.1583
Epoch 1914/10000
1/1 [==============================] - 0s 5ms/step - loss: 1922.0884
Epoch 1915/10000
1/1 [==============================] - 0s 4ms/step - loss: 1836.4648
Epoch 1916/10000
1/1 [==============================] - 0s 4ms/step - loss: 1006.9287
Epoch 1917/10000
1/1 [==============================] - 0s 5ms/step - loss: 885.0993
Epoch 1918/10000
1/1 [==============================] - 0s 4ms/step - loss: 2295.8879
Epoch 1919/10000
1/1 [==============================] - 0s 4ms/step - loss: 2488.0000
Epoch 1920/10000
1/1 [==============================] - 0s 4ms/step - loss: 1630.4263
Epoch 1921/10000
1/1 [==============================] - 0s 4ms/step - loss: 1391.3857
Epoch 1922/10000
1/1 [==============================] - 0s 4ms/step - loss: 1501.8940
Epoch 1923/10000
1/1 [==============================] - 0s 4ms/step - loss: 683.9461
Epoch 1924/10000
1/1 [==============================] - 0s 5ms/step - loss

1/1 [==============================] - 0s 4ms/step - loss: 1722.7941
Epoch 2009/10000
1/1 [==============================] - 0s 4ms/step - loss: 975.1636
Epoch 2010/10000
1/1 [==============================] - 0s 4ms/step - loss: 2348.3318
Epoch 2011/10000
1/1 [==============================] - 0s 4ms/step - loss: 2095.2122
Epoch 2012/10000
1/1 [==============================] - 0s 5ms/step - loss: 890.2712
Epoch 2013/10000
1/1 [==============================] - 0s 4ms/step - loss: 1459.8790
Epoch 2014/10000
1/1 [==============================] - 0s 5ms/step - loss: 579.7868
Epoch 2015/10000
1/1 [==============================] - 0s 4ms/step - loss: 2357.5942
Epoch 2016/10000
1/1 [==============================] - 0s 4ms/step - loss: 2001.0634
Epoch 2017/10000
1/1 [==============================] - 0s 5ms/step - loss: 1969.0586
Epoch 2018/10000
1/1 [==============================] - 0s 4ms/step - loss: 2475.1248
Epoch 2019/10000
1/1 [==============================] - 0s 4ms/step - loss

1/1 [==============================] - 0s 4ms/step - loss: 652.0017
Epoch 2104/10000
1/1 [==============================] - 0s 4ms/step - loss: 1867.1626
Epoch 2105/10000
1/1 [==============================] - 0s 5ms/step - loss: 1972.8309
Epoch 2106/10000
1/1 [==============================] - 0s 5ms/step - loss: 1738.7704
Epoch 2107/10000
1/1 [==============================] - 0s 4ms/step - loss: 1658.7374
Epoch 2108/10000
1/1 [==============================] - 0s 4ms/step - loss: 1115.6407
Epoch 2109/10000
1/1 [==============================] - 0s 5ms/step - loss: 2193.4124
Epoch 2110/10000
1/1 [==============================] - 0s 4ms/step - loss: 1666.6432
Epoch 2111/10000
1/1 [==============================] - 0s 4ms/step - loss: 2322.8984
Epoch 2112/10000
1/1 [==============================] - 0s 4ms/step - loss: 2350.0481
Epoch 2113/10000
1/1 [==============================] - 0s 4ms/step - loss: 1148.9957
Epoch 2114/10000
1/1 [==============================] - 0s 4ms/step - lo

1/1 [==============================] - 0s 6ms/step - loss: 3103.8601
Epoch 2199/10000
1/1 [==============================] - 0s 4ms/step - loss: 1305.9493
Epoch 2200/10000
1/1 [==============================] - 0s 4ms/step - loss: 2423.9919
Epoch 2201/10000
1/1 [==============================] - 0s 5ms/step - loss: 914.7285
Epoch 2202/10000
1/1 [==============================] - 0s 4ms/step - loss: 2164.9197
Epoch 2203/10000
1/1 [==============================] - 0s 4ms/step - loss: 1325.3649
Epoch 2204/10000
1/1 [==============================] - 0s 5ms/step - loss: 1788.9633
Epoch 2205/10000
1/1 [==============================] - 0s 4ms/step - loss: 1719.2363
Epoch 2206/10000
1/1 [==============================] - 0s 6ms/step - loss: 1063.8984
Epoch 2207/10000
1/1 [==============================] - 0s 6ms/step - loss: 693.3968
Epoch 2208/10000
1/1 [==============================] - 0s 5ms/step - loss: 1527.7861
Epoch 2209/10000
1/1 [==============================] - 0s 5ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 2708.1694
Epoch 2294/10000
1/1 [==============================] - 0s 5ms/step - loss: 2100.9958
Epoch 2295/10000
1/1 [==============================] - 0s 4ms/step - loss: 2475.9941
Epoch 2296/10000
1/1 [==============================] - 0s 4ms/step - loss: 793.9604
Epoch 2297/10000
1/1 [==============================] - 0s 5ms/step - loss: 2546.0354
Epoch 2298/10000
1/1 [==============================] - 0s 4ms/step - loss: 1231.3655
Epoch 2299/10000
1/1 [==============================] - 0s 4ms/step - loss: 1832.4205
Epoch 2300/10000
1/1 [==============================] - 0s 4ms/step - loss: 969.8530
Epoch 2301/10000
1/1 [==============================] - 0s 6ms/step - loss: 1411.9591
Epoch 2302/10000
1/1 [==============================] - 0s 6ms/step - loss: 2583.1907
Epoch 2303/10000
1/1 [==============================] - 0s 5ms/step - loss: 1652.2881
Epoch 2304/10000
1/1 [==============================] - 0s 16ms/step - lo

In [24]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 578ms/step
[610.155](test_target) - [[590.85706]](prediction) = 19.297944335937473


In [25]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.031627937714084904

In [26]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [27]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
0,-0.924122,-0.742263,0.825349,-0.569521,-1.283796,-1.243968,1.148205,-1.122043,-1.121418,-1.109924,...,-0.396211,-0.995947,-1.047594,0.868878,1.133056,1.331872,-1.066913,0.489423,-1.226856,1.235124
1,-0.465030,1.413621,-1.407209,-0.836281,0.128184,0.039396,0.140893,-0.184474,-0.185492,-0.204018,...,1.373802,-0.371548,-0.298948,0.531888,0.166368,-0.254125,-0.270454,0.904353,0.004233,-0.021028
2,1.389153,-0.671358,0.581860,1.405802,1.155611,1.204571,-1.289098,1.306518,1.306910,1.313942,...,-0.977591,1.367495,1.346542,-1.400766,-1.299425,-1.077747,1.337367,-1.393776,1.222623,-1.214095


train_target:


,Amazonas - Consumo de Cimento (t)
0,744.252
1,796.116
2,948.011


test_input:


...

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
12,0.548425,-0.389186,-0.34651,0.585399,-0.362055,0.429222,1.108405,2.635823,-0.189959,0.726405,...,0.598328,0.774435,0.860523,-1.989017,-1.013503,-0.547354,1.083075,-1.613773,0.941105,-1.177883


test_target:


,Amazonas - Consumo de Cimento (t)
12,610.155


1/1 [==============================] - 1s 566ms/step
Error: 19.297944335937473




[961.4811401367188,
 985.5841064453125,
 763.0487060546875,
 695.80126953125,
 756.9091796875,
 735.3483276367188,
 691.4617309570312,
 358.73834228515625,
 528.0046997070312,
 590.8570556640625]

In [28]:
display(targets)
display(predictions)

[994.932,
 1036.606,
 771.5678571428572,
 746.8674857142857,
 739.4576571428571,
 687.211,
 350.177,
 533.444,
 595.32,
 610.155]

[961.4811401367188,
 985.5841064453125,
 763.0487060546875,
 695.80126953125,
 756.9091796875,
 735.3483276367188,
 691.4617309570312,
 358.73834228515625,
 528.0046997070312,
 590.8570556640625]

In [29]:
mae = mean_absolute_error(predictions, targets)
mae

81.2250604171317

In [30]:
porcentage = mae/np.mean(targets)
porcentage

0.11495623021562885